In [39]:
%%configure -f
{
    "conf":{
        "spark.driver.maxResultSize":"6g"
    }
}

StatementMeta(sparkpool, 44, 3, Submitted, Available)

In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

StatementMeta(sparkpool, 44, 4, Submitted, Available)

In [41]:
# Primary storage info
account_name = 'calcuttaadlsgen2' # fill in your primary account name
container_name = 'yelp' # fill in your container name
relative_path_raw_json = 'raw_json' # fill in your relative folder path

adls_path_raw_json = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path_raw_json)
print('Primary storage account path: ' + adls_path_raw_json)

relative_path_staging_csv = 'staging_csv' # fill in your relative folder path

adls_path_staging_csv = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path_staging_csv)
print('Primary storage account path: ' + adls_path_staging_csv)

relative_path_staging_parquet = 'staging_parquet' # fill in your relative folder path

adls_path_staging_parquet = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path_staging_parquet)
print('Primary storage account path: ' + adls_path_staging_parquet)

StatementMeta(sparkpool, 44, 5, Finished, Available)

Primary storage account path: abfss://yelp@calcuttaadlsgen2.dfs.core.windows.net/raw_json
Primary storage account path: abfss://yelp@calcuttaadlsgen2.dfs.core.windows.net/staging_csv
Primary storage account path: abfss://yelp@calcuttaadlsgen2.dfs.core.windows.net/staging_parquet

## Description of tip data
### tip.json
### Tips written by a user on a business. Tips are shorter than reviews and tend to convey quick suggestions.

{
    
    // string, text of the tip
    "text": "Secret menu - fried chicken sando is da bombbbbbb Their zapatos are good too.",

    // string, when the tip was written, formatted like YYYY-MM-DD
    "date": "2013-09-20",

    // integer, how many compliments it has
    "compliment_count": 172,

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "49JhAJh8vSQ-vM4Aourl0g"
}

In [5]:
#Read the tip.json data
tip_file_name = '/yelp_academic_dataset_tip.json'
tip_file_path = adls_path_raw_json + tip_file_name
df_raw_tip_json = spark.read.json(tip_file_path)
print ("__________________Raw Schema__________________")
df_raw_tip_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_tip_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_tip_json.head(10))

StatementMeta(sparkpool, 40, 6, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

1320761
__________________Raw Sample__________________

[Row(business_id='UYX5zL_Xj9WEc_Wp-FrqHw', compliment_count=0, date='2013-11-26 18:20:08', text='Here for a quick mtg', user_id='hf27xTME3EiCp6NL6VtWZQ'), Row(business_id='Ch3HkwQYv1YKw_FO06vBWA', compliment_count=0, date='2014-06-15 22:26:45', text='Cucumber strawberry refresher', user_id='uEvusDwoSymbJJ0auR3muQ'), Row(business_id='rDoT-MgxGRiYqCmi0bG10g', compliment_count=0, date='2016-07-18 22:03:42', text='Very nice good service good food', user_id='AY-laIws3S7YXNl_f_D6rQ'), Row(business_id='OHXnDV01gLokiX1ELaQufA', compliment_count=0, date='2014-06-06 01:10:34', text="It's a small place. The staff is friendly.", user_id='Ue_7yUlkEb

In [9]:
#Read the tip.json data by attaching the custom schema
from pyspark.sql.types import StructField, StructType,StringType,LongType,IntegerType,DateType
custom_schem_tips_staging = StructType([
    StructField("business_id", StringType(), True, metadata={"desc": "string, 22 character business id, maps to business in business.json"}),
    StructField("compliment_count", IntegerType(), True, metadata={"desc": "integer, how many compliments it has"}),
    StructField("date", DateType(), True, metadata={"desc": "string, when the tip was written, formatted like YYYY-MM-DD HH:MI:SS"}),
    StructField("text", StringType(), True, metadata={"desc": "string, text of the tip"}),
    StructField("user_id", StringType(), True, metadata={"desc": "string, 22 character unique user id, maps to the user in user.json"})
])

tip_file_name = '/yelp_academic_dataset_tip.json'
tip_file_path = adls_path_raw_json + tip_file_name
#df_raw_tip_json = spark.read.json(tip_file_path)

df_raw_tip_json = spark.read.format("json") \
    .schema(custom_schem_tips_staging) \
    .option("header", True) \
    .load(tip_file_path)


print ("__________________Raw Schema__________________")
df_raw_tip_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_tip_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_tip_json.head(10))

StatementMeta(sparkpool, 40, 10, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- compliment_count: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

1320761
__________________Raw Sample__________________

[Row(business_id='UYX5zL_Xj9WEc_Wp-FrqHw', compliment_count=0, date=datetime.date(2013, 11, 26), text='Here for a quick mtg', user_id='hf27xTME3EiCp6NL6VtWZQ'), Row(business_id='Ch3HkwQYv1YKw_FO06vBWA', compliment_count=0, date=datetime.date(2014, 6, 15), text='Cucumber strawberry refresher', user_id='uEvusDwoSymbJJ0auR3muQ'), Row(business_id='rDoT-MgxGRiYqCmi0bG10g', compliment_count=0, date=datetime.date(2016, 7, 18), text='Very nice good service good food', user_id='AY-laIws3S7YXNl_f_D6rQ'), Row(business_id='OHXnDV01gLokiX1ELaQufA', compliment_count=0, date=datetime.date(2014, 6, 6), text="It's a small place. The staff is friendly."

In [12]:
#Write the tip.json data in parquet format with the custom schema
tip_file_parquet_staging_path = adls_path_staging_parquet + '/yelp_tips_parquet'
df_raw_tip_json.coalesce(1).write.parquet(tip_file_parquet_staging_path, mode = 'overwrite')
df_staging_tip_parquet = spark.read.parquet(tip_file_parquet_staging_path)
print ("__________________Raw Schema__________________")
df_staging_tip_parquet.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_tip_parquet.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_tip_parquet.head(10))
df_staging_tip_parquet.show(5, truncate = False)

StatementMeta(sparkpool, 40, 13, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- compliment_count: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

1320761


__________________Raw Sample__________________

[Row(business_id='UYX5zL_Xj9WEc_Wp-FrqHw', compliment_count=0, date=datetime.date(2013, 11, 26), text='Here for a quick mtg', user_id='hf27xTME3EiCp6NL6VtWZQ'), Row(business_id='Ch3HkwQYv1YKw_FO06vBWA', compliment_count=0, date=datetime.date(2014, 6, 15), text='Cucumber strawberry refresher', user_id='uEvusDwoSymbJJ0auR3muQ'), Row(business_id='rDoT-MgxGRiYqCmi0bG10g', compliment_count=0, date=datetime.date(2016, 7, 18), text='Very nice good service good food', user_id='AY-laIws3S7YXNl_f_D6rQ'), Row(business_id='OHXnDV01gLokiX1ELaQufA', compliment_count=0, date=datetime.date(2014, 6, 6), text="It's a small place. The staff is friendly

In [48]:
#Write the tip.json data in csv format
tip_file_csv_staging_path = adls_path_staging_csv + '/yelp_tips_csv'
df_raw_tip_json.coalesce(1).write.csv(tip_file_csv_staging_path, mode = 'overwrite', header = 'true')
df_staging_tip_csv = spark.read.csv(tip_file_csv_staging_path, header = 'true')
print ("__________________Raw Schema__________________")
df_staging_tip_csv.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_tip_csv.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_tip_csv.head(10))
df_staging_tip_csv.show(5, truncate = False)

StatementMeta(sparkpool, 27, 43, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- compliment_count: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

1363162


__________________Raw Sample__________________

[Row(business_id='UYX5zL_Xj9WEc_Wp-FrqHw', compliment_count='0', date='2013-11-26 18:20:08', text='Here for a quick mtg', user_id='hf27xTME3EiCp6NL6VtWZQ'), Row(business_id='Ch3HkwQYv1YKw_FO06vBWA', compliment_count='0', date='2014-06-15 22:26:45', text='Cucumber strawberry refresher', user_id='uEvusDwoSymbJJ0auR3muQ'), Row(business_id='rDoT-MgxGRiYqCmi0bG10g', compliment_count='0', date='2016-07-18 22:03:42', text='Very nice good service good food', user_id='AY-laIws3S7YXNl_f_D6rQ'), Row(business_id='OHXnDV01gLokiX1ELaQufA', compliment_count='0', date='2014-06-06 01:10:34', text="It's a small place. The staff is friendly.", user_id

In [30]:
#JUNK
# Display 5 rows
#df_raw_tip_json.show(5, truncate = False)
df_staging_tip_csv.describe()

StatementMeta(sparkpool, 27, 25, Finished, Available)

DataFrame[summary: string, business_id: string, compliment_count: string, date: string, text: string, user_id: string]

In [49]:
from pyspark.sql.types import StructField, StructType, StringType,LongType
custom_schem_tips_staging = StructType([
    StructField("business_id", StringType(), True, metadata={"desc": "string, 22 character business id, maps to business in business.json"}),
    StructField("compliment_count", LongType(), True, metadata={"desc": "integer, how many compliments it has"}),
    StructField("date", StringType(), True, metadata={"desc": "string, when the tip was written, formatted like YYYY-MM-DD HH:MI:SS"}),
    StructField("text", StringType(), True, metadata={"desc": "string, text of the tip"}),
    StructField("user_id", StringType(), True, metadata={"desc": "string, 22 character unique user id, maps to the user in user.json"})
])
df = spark.read.format("csv") \
    .schema(custom_schem_tips_staging) \
    .option("header", True) \
    .load(tip_file_csv_staging_path)
df.printSchema()    
df.show(5, truncate = False)
df.schema.json()
#df.schema.fields[0].metadata["desc"]

StatementMeta(sparkpool, 27, 44, Finished, Available)

root
 |-- business_id: string (nullable = true)
 |-- compliment_count: long (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

+----------------------+----------------+-------------------+-----------------------------------------------------------------------------+----------------------+
|business_id           |compliment_count|date               |text                                                                         |user_id               |
+----------------------+----------------+-------------------+-----------------------------------------------------------------------------+----------------------+
|UYX5zL_Xj9WEc_Wp-FrqHw|0               |2013-11-26 18:20:08|Here for a quick mtg                                                         |hf27xTME3EiCp6NL6VtWZQ|
|Ch3HkwQYv1YKw_FO06vBWA|0               |2014-06-15 22:26:45|Cucumber strawberry refresher                                                |uEv

## Description of checkin data
### checkin.json
### Checkins on a business.

{

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg"

    // string which is a comma-separated list of timestamps for each checkin, each with format YYYY-MM-DD HH:MM:SS
    "date": "2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02"
}

In [50]:
#Read the checkin.json data
checkin_file_name = '/yelp_academic_dataset_checkin.json'
checkin_file_path = adls_path_raw_json + checkin_file_name
df_raw_checkin_json = spark.read.json(checkin_file_path)
print ("__________________Raw Schema__________________")
df_raw_checkin_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_checkin_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_checkin_json.head(10))

StatementMeta(sparkpool, 27, 45, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)

__________________Raw Count__________________

175187
__________________Raw Sample__________________

[Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02, 2019-03-19 22:04:48'), Row(business_id='--6MefnULPED_I942VcFNA', date='2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012-04-15 01:07:50, 2012-05-06 23:08:42, 2012-06-08 22:43:12, 2012-08-06 23:20:52, 2012-08-19 18:30:44, 2013-01-27 23:49:51, 2013-03-01 01:22:29, 2013-03-23 21:53:47, 2013-03-24 01:11:51, 2013-05-20 00:12:25, 2013-06-29 22:50:57, 2013-07-01 15:58:04, 2013-07-01 22:35:42, 2013-07-28 21:46:49, 2013-08-02 22:06:03, 2013-08-11 18:42:03, 2013-08-23 01:29:56, 2013-09-17 17:40:36, 2013-09-20 23:36:30, 2013-09-24 23:50:18, 2013-10-12 23:11:13, 2013-11-24 18:40:15, 2013-12-12 02:52

In [58]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
display(df_raw_checkin_json.select("business_id",explode(split(col("date"), ",\s*")).alias("date")))

StatementMeta(sparkpool, 27, 53, Finished, Available)

SynapseWidget(Synapse.DataFrame, 45287e12-a178-4a95-acc3-b69f4bc38bc8)

In [59]:
#Write the checkin.json data in csv format
checkin_file_csv_staging_path = adls_path_staging_csv + '/yelp_checkins_csv'
df_raw_checkin_json_exploded = df_raw_checkin_json.select("business_id",explode(split(col("date"), ",\s*")).alias("date"))
df_raw_checkin_json_exploded.coalesce(4).write.csv(checkin_file_csv_staging_path, mode = 'overwrite', header = 'true')
df_staging_checkin_csv = spark.read.csv(checkin_file_csv_staging_path, header = 'true')
print ("__________________Raw Schema__________________")
df_staging_checkin_csv.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_checkin_csv.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_checkin_csv.head(10))
df_staging_checkin_csv.show(5, truncate = False)

StatementMeta(sparkpool, 27, 54, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)

__________________Raw Count__________________

21003777


__________________Raw Sample__________________

[Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-04-26 19:49:16'), Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-08-30 18:36:57'), Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-10-15 02:45:18'), Row(business_id='--1UhMGODdWsrMastO9DZw', date='2016-11-18 01:54:50'), Row(business_id='--1UhMGODdWsrMastO9DZw', date='2017-04-20 18:39:06'), Row(business_id='--1UhMGODdWsrMastO9DZw', date='2017-05-03 17:58:02'), Row(business_id='--1UhMGODdWsrMastO9DZw', date='2019-03-19 22:04:48'), Row(business_id='--6MefnULPED_I942VcFNA', date='2011-06-04 18:22:23'), Row(business_id='--6MefnULPED_I942VcFNA', date='2011-07-23 23:51:33'), Row(business_id='--6MefnULPED_I942VcFNA', date='2012-04-15 01:07:50')]
+----------------------+-------------------+
|busin

## Description of review data
### review.json
### Contains full review text data including the user_id that wrote the review and the business_id the review is written for.

{
    
    // string, 22 character unique review id
    "review_id": "zdSx_SD6obEhz9VrW9uAWA",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // integer, star rating
    "stars": 4,

    // string, date formatted YYYY-MM-DD
    "date": "2016-03-09",

    // string, the review itself
    "text": "Great place to hang out after work: the prices are decent, and the ambience is fun. It's a bit loud, but very lively. The staff is friendly, and the food is good. They have a good selection of drinks.",

    // integer, number of useful votes received
    "useful": 0,

    // integer, number of funny votes received
    "funny": 0,

    // integer, number of cool votes received
    "cool": 0
}

In [22]:
#Read the review.json data
review_file_name = '/yelp_academic_dataset_review.json'
review_file_path = adls_path_raw_json + review_file_name
df_raw_review_json = spark.read.json(review_file_path)
print ("__________________Raw Schema__________________")
df_raw_review_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_review_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_review_json.head(10))

StatementMeta(sparkpool, 42, 6, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

8021122
__________________Raw Sample__________________

[Row(business_id='-MhfebM0QIsKt87iDN-FNw', cool=0, date='2015-04-15 05:21:16', funny=0, review_id='xQY8N_XvtGbearJ5X4QryQ', stars=2.0, text='As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It\'s what real es

In [31]:
#Read the review.json data by attaching the custom schema
from pyspark.sql.types import StructField, StructType,StringType,LongType,IntegerType,DateType,FloatType
custom_schema_review_staging = StructType([
	StructField("review_id", StringType(), True, metadata={"desc": "string, 22 character review_id"}),
    StructField("business_id", StringType(), True, metadata={"desc": "string, 22 character business id, maps to business in business.json"}),
    StructField("stars", FloatType(), True, metadata={"desc": "integer, star rating"}),
	StructField("useful", IntegerType(), True, metadata={"desc": "integer, number of useful votes received"}),
	StructField("funny", IntegerType(), True, metadata={"desc": "integer, number of funny votes received"}),
	StructField("cool", IntegerType(), True, metadata={"desc": "integer, number of cool votes received"}),
    StructField("date", DateType(), True, metadata={"desc": "string, when the tip was written, formatted like YYYY-MM-DD HH:MI:SS"}),
    StructField("text", StringType(), True, metadata={"desc": "string, text of the tip"}),
    StructField("user_id", StringType(), True, metadata={"desc": "string, 22 character unique user id, maps to the user in user.json"})
])

review_file_name = '/yelp_academic_dataset_review.json'
review_file_path = adls_path_raw_json + review_file_name
df_raw_review_json = spark.read.format("json") \
    .schema(custom_schema_review_staging) \
    .option("header", True) \
    .load(review_file_path)

print ("__________________Raw Schema__________________")
df_raw_review_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_review_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_review_json.head(10))

StatementMeta(sparkpool, 43, 6, Finished, Available)

__________________Raw Schema__________________
root
 |-- review_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: float (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- cool: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

8021122
__________________Raw Sample__________________

[Row(review_id='xQY8N_XvtGbearJ5X4QryQ', business_id='-MhfebM0QIsKt87iDN-FNw', stars=2.0, useful=5, funny=0, cool=0, date=datetime.date(2015, 4, 15), text='As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from vie

In [34]:
df_raw_review_json = df_raw_review_json.withColumn("stars",df_raw_review_json["stars"].cast(IntegerType()))
print ("__________________Raw Schema__________________")
df_raw_review_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_review_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_review_json.head(10))

StatementMeta(sparkpool, 43, 9, Finished, Available)

__________________Raw Schema__________________
root
 |-- review_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- cool: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

8021122
__________________Raw Sample__________________

[Row(review_id='xQY8N_XvtGbearJ5X4QryQ', business_id='-MhfebM0QIsKt87iDN-FNw', stars=2, useful=5, funny=0, cool=0, date=datetime.date(2015, 4, 15), text='As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from vie

In [38]:
#Write the review.json data in parquet format
review_file_parquet_staging_path = adls_path_staging_parquet + '/yelp_reviews_parquet'
df_raw_review_json.coalesce(25).write.parquet(review_file_parquet_staging_path, mode = 'overwrite')
df_staging_review_parquet = spark.read.parquet(review_file_parquet_staging_path)
print ("__________________Raw Schema__________________")
df_staging_review_parquet.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_review_parquet.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_review_parquet.head(10))
df_staging_review_parquet.show(5, truncate = False)

StatementMeta(sparkpool, 43, 13, Finished, Available)

__________________Raw Schema__________________
root
 |-- review_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- useful: integer (nullable = true)
 |-- funny: integer (nullable = true)
 |-- cool: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

8021122


__________________Raw Sample__________________

[Row(review_id='r4CwXVns8GRdZ2qe0GRrhg', business_id='3FLFxKPE76UCHk0TiBzozQ', stars=5, useful=0, funny=1, cool=1, date=datetime.date(2013, 10, 17), text="Great strip club. Very debonair. I'm vip host for 15 strip clubs on or near the Las Vegas Strip. I can get free transportation the horse with a waived cover charge. You just have to buy 2 drink minimum for $33 and I'll add a free drink card to give a total of 3 drinks. My name is Teddie, give me a call at 702-858-8487 or 702-858-TITS. I hav

In [61]:
#Write the review.json data in csv format
review_file_csv_staging_path = adls_path_staging_csv + '/yelp_reviews_csv'
df_raw_review_json.coalesce(25).write.csv(review_file_csv_staging_path, mode = 'overwrite', header = 'true')
df_staging_review_csv = spark.read.csv(review_file_csv_staging_path, header = 'true')
print ("__________________Raw Schema__________________")
df_staging_review_csv.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_review_csv.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_review_csv.head(10))
df_staging_review_csv.show(5, truncate = False)

StatementMeta(sparkpool, 27, 56, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- cool: string (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- user_id: string (nullable = true)

__________________Raw Count__________________

17985307


__________________Raw Sample__________________

[Row(business_id='3FLFxKPE76UCHk0TiBzozQ', cool='1', date='2013-10-17 23:41:13', funny='1', review_id='r4CwXVns8GRdZ2qe0GRrhg', stars='5.0', text="Great strip club. Very debonair. I'm vip host for 15 strip clubs on or near the Las Vegas Strip. I can get free transportation the horse with a waived cover charge. You just have to buy 2 drink minimum for $33 and I'll add a free drink card to give a total of 3 drinks. My name is Teddie, give me a call at 702-858-8487 or 702-858-TITS. I have the tic

## Description of user data
### user.json
### User data including the user's friend mapping and all the metadata associated with the user.

{
    
    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, the user's first name
    "name": "Sebastien",

    // integer, the number of reviews they've written
    "review_count": 56,

    // string, when the user joined Yelp, formatted like YYYY-MM-DD
    "yelping_since": "2011-01-01",

    // array of strings, an array of the user's friend as user_ids
    "friends": [
        "wqoXYLWmpkEH0YvTmHBsJQ",
        "KUXLLiJGrjtSsapmxmpvTA",
        "6e9rJKQC3n0RSKyHLViL-Q"
    ],

    // integer, number of useful votes sent by the user
    "useful": 21,

    // integer, number of funny votes sent by the user
    "funny": 88,

    // integer, number of cool votes sent by the user
    "cool": 15,

    // integer, number of fans the user has
    "fans": 1032,

    // array of integers, the years the user was elite
    "elite": [
        2012,
        2013
    ],

    // float, average rating of all reviews
    "average_stars": 4.31,

    // integer, number of hot compliments received by the user
    "compliment_hot": 339,

    // integer, number of more compliments received by the user
    "compliment_more": 668,

    // integer, number of profile compliments received by the user
    "compliment_profile": 42,

    // integer, number of cute compliments received by the user
    "compliment_cute": 62,

    // integer, number of list compliments received by the user
    "compliment_list": 37,

    // integer, number of note compliments received by the user
    "compliment_note": 356,

    // integer, number of plain compliments received by the user
    "compliment_plain": 68,

    // integer, number of cool compliments received by the user
    "compliment_cool": 91,

    // integer, number of funny compliments received by the user
    "compliment_funny": 99,

    // integer, number of writer compliments received by the user
    "compliment_writer": 95,

    // integer, number of photo compliments received by the user
    "compliment_photos": 50
}

In [62]:
#Read the user.json data
user_file_name = '/yelp_academic_dataset_user.json'
user_file_path = adls_path_raw_json + user_file_name
df_raw_user_json = spark.read.json(user_file_path)
print ("__________________Raw Schema__________________")
df_raw_user_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_user_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_user_json.head(10))

StatementMeta(sparkpool, 27, 57, Finished, Available)

__________________Raw Schema__________________
root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

__________________Raw 

In [63]:
#Write the user.json data in csv format
user_file_csv_staging_path = adls_path_staging_csv + '/yelp_users_csv'
df_raw_user_json.coalesce(15).write.csv(user_file_csv_staging_path, mode = 'overwrite', header = 'true')
df_staging_user_csv = spark.read.csv(user_file_csv_staging_path, header = 'true')
print ("__________________Raw Schema__________________")
df_staging_user_csv.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_user_csv.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_user_csv.head(10))
df_staging_user_csv.show(5, truncate = False)

StatementMeta(sparkpool, 27, 58, Finished, Available)

__________________Raw Schema__________________
root
 |-- average_stars: string (nullable = true)
 |-- compliment_cool: string (nullable = true)
 |-- compliment_cute: string (nullable = true)
 |-- compliment_funny: string (nullable = true)
 |-- compliment_hot: string (nullable = true)
 |-- compliment_list: string (nullable = true)
 |-- compliment_more: string (nullable = true)
 |-- compliment_note: string (nullable = true)
 |-- compliment_photos: string (nullable = true)
 |-- compliment_plain: string (nullable = true)
 |-- compliment_profile: string (nullable = true)
 |-- compliment_writer: string (nullable = true)
 |-- cool: string (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: string (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable

## Description of business data
### business.json
### Contains business data including location data, attributes, and categories. *
### *We will separate the business data into multiple tables -> business, attributes, hours
{
    
    // string, 22 character unique string business id
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, the business's name
    "name": "Garaje",

    // string, the full address of the business
    "address": "475 3rd St",

    // string, the city
    "city": "San Francisco",

    // string, 2 character state code, if applicable
    "state": "CA",

    // string, the postal code
    "postal code": "94107",

    // float, latitude
    "latitude": 37.7817529521,

    // float, longitude
    "longitude": -122.39612197,

    // float, star rating, rounded to half-stars
    "stars": 4.5,

    // integer, number of reviews
    "review_count": 1198,

    // integer, 0 or 1 for closed or open, respectively
    "is_open": 1,

    // object, business attributes to values. note: some attribute values might be objects
    "attributes": {
        "RestaurantsTakeOut": true,
        "BusinessParking": {
            "garage": false,
            "street": true,
            "validated": false,
            "lot": false,
            "valet": false
        },
    },

    // an array of strings of business categories
    "categories": [
        "Mexican",
        "Burgers",
        "Gastropubs"
    ],

    // an object of key day to value hours, hours are using a 24hr clock
    "hours": {
        "Monday": "10:00-21:00",
        "Tuesday": "10:00-21:00",
        "Friday": "10:00-21:00",
        "Wednesday": "10:00-21:00",
        "Thursday": "10:00-21:00",
        "Sunday": "11:00-18:00",
        "Saturday": "10:00-21:00"
    }
}

In [43]:
#Read the business.json data
business_file_name = '/yelp_academic_dataset_business.json'
business_file_path = adls_path_raw_json + business_file_name
df_raw_business_json = spark.read.json(business_file_path)
print ("__________________Raw Schema__________________")
df_raw_business_json.printSchema()
print ("__________________Raw Count__________________\n")
print(df_raw_business_json.count())
print ("__________________Raw Sample__________________\n")
print(df_raw_business_json.head(10))

StatementMeta(sparkpool, 44, 7, Finished, Available)

__________________Raw Schema__________________
root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string 

In [37]:
#Write the business.json data in parquet format
business_file_parquet_staging_path = adls_path_staging_parquet + '/yelp_businesss_parquet'
df_raw_business_json_droped_columns = df_raw_business_json.drop("attributes","hours")
df_raw_business_json_droped_columns.coalesce(1).write.parquet(business_file_parquet_staging_path, mode = 'overwrite')
df_staging_business_parquet = spark.read.parquet(business_file_parquet_staging_path)
print ("__________________Raw Schema__________________")
df_staging_business_parquet.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_business_parquet.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_business_parquet.head(10))
df_staging_business_parquet.show(5, truncate = False)

StatementMeta(sparkpool, 43, 12, Finished, Available)

__________________Raw Schema__________________
root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)

__________________Raw Count__________________

209393


__________________Raw Sample__________________

[Row(address='10913 Bailey Rd', business_id='f9NumwFMBDn751xgFiRbNA', categories='Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping', city='Cornelius', is_open=1, latitude=35.4627242, longitude=-80.8526119, name='The Range At Lake Norman', postal_code='28031', review_count=36, stars=3.5, state='NC'), Row(address='8880 E Via Linda, Ste 107', business_id='Yzvjg0Sayho

In [65]:
#Write the business.json data in csv format
business_file_csv_staging_path = adls_path_staging_csv + '/yelp_businesss_csv'
df_raw_business_json_droped_columns = df_raw_business_json.drop("attributes","hours")
df_raw_business_json_droped_columns.coalesce(1).write.csv(business_file_csv_staging_path, mode = 'overwrite', header = 'true')
df_staging_business_csv = spark.read.csv(business_file_csv_staging_path, header = 'true')
print ("__________________Raw Schema__________________")
df_staging_business_csv.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_business_csv.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_business_csv.head(10))
df_staging_business_csv.show(5, truncate = False)

StatementMeta(sparkpool, 27, 60, Finished, Available)

__________________Raw Schema__________________
root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- state: string (nullable = true)

__________________Raw Count__________________

209393


__________________Raw Sample__________________

[Row(address='10913 Bailey Rd', business_id='f9NumwFMBDn751xgFiRbNA', categories='Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping', city='Cornelius', is_open='1', latitude='35.4627242', longitude='-80.8526119', name='The Range At Lake Norman', postal_code='28031', review_count='36', stars='3.5', state='NC'), Row(address='8880 E Via Linda, Ste 107', business_i

In [74]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
#JUNK
display(df_raw_business_json.select("business_id",col("hours.Friday").alias("Friday")))

StatementMeta(sparkpool, 28, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8d035575-9a5e-42eb-8315-b0fd9bf9ffdf)

In [78]:
#JUNK
display(df_raw_business_json.select("business_id",col("hours.Friday").alias("Friday"),col("hours.Saturday").alias("Saturday"),col("hours.Sunday").alias("Sunday"),col("hours.Monday").alias("Monday"),col("hours.Tuesday").alias("Tuesday"),col("hours.Wednesday").alias("Wednesday"),col("hours.Thursday").alias("Thursday")))

StatementMeta(sparkpool, 28, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 25f00c8d-a9c7-41f1-ab72-4988114965f5)

In [45]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
#Write the business_hours.json data in parquet format
df_raw_business_json_select_hours = df_raw_business_json.select("business_id",col("hours.Friday").alias("Friday"),col("hours.Saturday").alias("Saturday"),col("hours.Sunday").alias("Sunday"),col("hours.Monday").alias("Monday"),col("hours.Tuesday").alias("Tuesday"),col("hours.Wednesday").alias("Wednesday"),col("hours.Thursday").alias("Thursday"))
#df_raw_business_json_select_hours.printSchema()
#df_raw_business_json_select_hours.show(5, truncate = False)

business_hours_file_parquet_staging_path = adls_path_staging_parquet + '/yelp_business_hours_parquet'
df_raw_business_json_select_hours.coalesce(1).write.parquet(business_hours_file_parquet_staging_path, mode = 'overwrite')
df_staging_business_hours_parquet = spark.read.parquet(business_hours_file_parquet_staging_path)
print ("__________________Raw Schema__________________")
df_staging_business_hours_parquet.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_business_hours_parquet.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_business_hours_parquet.head(10))
df_staging_business_hours_parquet.show(5, truncate = False)

StatementMeta(sparkpool, 44, 9, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
 |-- Thursday: string (nullable = true)

__________________Raw Count__________________

209393


__________________Raw Sample__________________

[Row(business_id='f9NumwFMBDn751xgFiRbNA', Friday='11:0-20:0', Saturday='11:0-20:0', Sunday='13:0-18:0', Monday='10:0-18:0', Tuesday='11:0-20:0', Wednesday='10:0-18:0', Thursday='11:0-20:0'), Row(business_id='Yzvjg0SayhoZgCljUJRF9Q', Friday=None, Saturday=None, Sunday=None, Monday=None, Tuesday=None, Wednesday=None, Thursday=None), Row(business_id='XNoUzKckATkOD1hP6vghZg', Friday=None, Saturday=None, Sunday=None, Monday=None, Tuesday=None, Wednesday=None, Thursday=None), Row(business_id='6OAZjbxqM5ol29BuHsil3w', Friday='

In [81]:
#Write the business_hours.json data in csv format
#df_raw_business_json_select_hours = df_raw_business_json.select("business_id",col("hours.Friday").alias("Friday"),col("hours.Saturday").alias("Saturday"),col("hours.Sunday").alias("Sunday"),col("hours.Monday").alias("Monday"),col("hours.Tuesday").alias("Tuesday"),col("hours.Wednesday").alias("Wednesday"),col("hours.Thursday").alias("Thursday"))
#df_raw_business_json_select_hours.printSchema()
#df_raw_business_json_select_hours.show(5, truncate = False)

business_hours_file_csv_staging_path = adls_path_staging_csv + '/yelp_business_hours_csv'
df_raw_business_json_select_hours.coalesce(1).write.csv(business_hours_file_csv_staging_path, mode = 'overwrite', header = 'true')
df_staging_business_hours_csv = spark.read.csv(business_hours_file_csv_staging_path, header = 'true')
print ("__________________Raw Schema__________________")
df_staging_business_hours_csv.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_business_hours_csv.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_business_hours_csv.head(10))
df_staging_business_hours_csv.show(5, truncate = False)

StatementMeta(sparkpool, 28, 15, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
 |-- Thursday: string (nullable = true)

__________________Raw Count__________________

209393


__________________Raw Sample__________________

[Row(business_id='f9NumwFMBDn751xgFiRbNA', Friday='11:0-20:0', Saturday='11:0-20:0', Sunday='13:0-18:0', Monday='10:0-18:0', Tuesday='11:0-20:0', Wednesday='10:0-18:0', Thursday='11:0-20:0'), Row(business_id='Yzvjg0SayhoZgCljUJRF9Q', Friday=None, Saturday=None, Sunday=None, Monday=None, Tuesday=None, Wednesday=None, Thursday=None), Row(business_id='XNoUzKckATkOD1hP6vghZg', Friday=None, Saturday=None, Sunday=None, Monday=None, Tuesday=None, Wednesday=None, Thursday=None), Row(business_id='6OAZjbxqM5ol29BuHsil3w', Friday='

In [46]:
#Write the business_attributes.json data in parquet format
business_attributes_file_parquet_staging_path = adls_path_staging_parquet + '/yelp_business_attributes_parquet'
df_raw_business_attributes_json = df_raw_business_json.select("business_id", \
col("attributes.AcceptsInsurance").alias("AcceptsInsurance"), \
col("attributes.AgesAllowed").alias("AgesAllowed"), \
col("attributes.Alcohol").alias("Alcohol"), \
col("attributes.Ambience").alias("Ambience"), \
col("attributes.BYOB").alias("BYOB"), \
col("attributes.BYOBCorkage").alias("BYOBCorkage"), \
col("attributes.BestNights").alias("BestNights"), \
col("attributes.BikeParking").alias("BikeParking"), \
col("attributes.BusinessAcceptsBitcoin").alias("BusinessAcceptsBitcoin"), \
col("attributes.BusinessAcceptsCreditCards").alias("BusinessAcceptsCreditCards"), \
#col("attributes.BusinessParking").alias("BusinessParking"), \
col("attributes.ByAppointmentOnly").alias("ByAppointmentOnly"), \
col("attributes.Caters").alias("Caters"), \
col("attributes.CoatCheck").alias("CoatCheck"), \
col("attributes.Corkage").alias("Corkage"), \
col("attributes.DietaryRestrictions").alias("DietaryRestrictions"), \
col("attributes.DogsAllowed").alias("DogsAllowed"), \
col("attributes.DriveThru").alias("DriveThru"), \
col("attributes.GoodForDancing").alias("GoodForDancing"), \
col("attributes.GoodForKids").alias("GoodForKids"), \
col("attributes.GoodForMeal").alias("GoodForMeal"), \
col("attributes.HairSpecializesIn").alias("HairSpecializesIn"), \
col("attributes.HappyHour").alias("HappyHour"), \
col("attributes.HasTV").alias("HasTV"), \
col("attributes.Music").alias("Music"), \
col("attributes.NoiseLevel").alias("NoiseLevel"), \
col("attributes.Open24Hours").alias("Open24Hours"), \
col("attributes.OutdoorSeating").alias("OutdoorSeating"), \
col("attributes.RestaurantsAttire").alias("RestaurantsAttire"), \
col("attributes.RestaurantsCounterService").alias("RestaurantsCounterService"), \
col("attributes.RestaurantsDelivery").alias("RestaurantsDelivery"), \
col("attributes.RestaurantsGoodForGroups").alias("RestaurantsGoodForGroups"), \
col("attributes.RestaurantsPriceRange2").alias("RestaurantsPriceRange2"), \
col("attributes.RestaurantsReservations").alias("RestaurantsReservations"), \
col("attributes.RestaurantsTableService").alias("RestaurantsTableService"), \
col("attributes.RestaurantsTakeOut").alias("RestaurantsTakeOut"), \
col("attributes.Smoking").alias("Smoking"), \
col("attributes.WheelchairAccessible").alias("WheelchairAccessible"), \
col("attributes.WiFi").alias("WiFi"), \
split(col("attributes.BusinessParking"), ":|,").getItem(1).alias("BusinessParking_garage"), \
split(col("attributes.BusinessParking"), ":|,").getItem(3).alias("BusinessParking_street"), \
split(col("attributes.BusinessParking"), ":|,").getItem(5).alias("BusinessParking_validated"), \
split(col("attributes.BusinessParking"), ":|,").getItem(7).alias("BusinessParking_lot"), \
split(col("attributes.BusinessParking"), ":|,|\}").getItem(9).alias("BusinessParking_valet"))
df_raw_business_attributes_json.coalesce(1).write.parquet(business_attributes_file_parquet_staging_path, mode = 'overwrite')
df_staging_business_attributes_parquet = spark.read.parquet(business_attributes_file_parquet_staging_path)
print ("__________________Raw Schema__________________")
df_staging_business_attributes_parquet.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_business_attributes_parquet.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_business_attributes_parquet.head(10))
df_staging_business_attributes_parquet.show(5, truncate = False)

StatementMeta(sparkpool, 44, 10, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: s

In [99]:
#Write the business_attributes.json data in csv format
business_attributes_file_csv_staging_path = adls_path_staging_csv + '/yelp_business_attributes_csv'
df_raw_business_attributes_json = df_raw_business_json.select("business_id", \
col("attributes.AcceptsInsurance").alias("AcceptsInsurance"), \
col("attributes.AgesAllowed").alias("AgesAllowed"), \
col("attributes.Alcohol").alias("Alcohol"), \
col("attributes.Ambience").alias("Ambience"), \
col("attributes.BYOB").alias("BYOB"), \
col("attributes.BYOBCorkage").alias("BYOBCorkage"), \
col("attributes.BestNights").alias("BestNights"), \
col("attributes.BikeParking").alias("BikeParking"), \
col("attributes.BusinessAcceptsBitcoin").alias("BusinessAcceptsBitcoin"), \
col("attributes.BusinessAcceptsCreditCards").alias("BusinessAcceptsCreditCards"), \
#col("attributes.BusinessParking").alias("BusinessParking"), \
col("attributes.ByAppointmentOnly").alias("ByAppointmentOnly"), \
col("attributes.Caters").alias("Caters"), \
col("attributes.CoatCheck").alias("CoatCheck"), \
col("attributes.Corkage").alias("Corkage"), \
col("attributes.DietaryRestrictions").alias("DietaryRestrictions"), \
col("attributes.DogsAllowed").alias("DogsAllowed"), \
col("attributes.DriveThru").alias("DriveThru"), \
col("attributes.GoodForDancing").alias("GoodForDancing"), \
col("attributes.GoodForKids").alias("GoodForKids"), \
col("attributes.GoodForMeal").alias("GoodForMeal"), \
col("attributes.HairSpecializesIn").alias("HairSpecializesIn"), \
col("attributes.HappyHour").alias("HappyHour"), \
col("attributes.HasTV").alias("HasTV"), \
col("attributes.Music").alias("Music"), \
col("attributes.NoiseLevel").alias("NoiseLevel"), \
col("attributes.Open24Hours").alias("Open24Hours"), \
col("attributes.OutdoorSeating").alias("OutdoorSeating"), \
col("attributes.RestaurantsAttire").alias("RestaurantsAttire"), \
col("attributes.RestaurantsCounterService").alias("RestaurantsCounterService"), \
col("attributes.RestaurantsDelivery").alias("RestaurantsDelivery"), \
col("attributes.RestaurantsGoodForGroups").alias("RestaurantsGoodForGroups"), \
col("attributes.RestaurantsPriceRange2").alias("RestaurantsPriceRange2"), \
col("attributes.RestaurantsReservations").alias("RestaurantsReservations"), \
col("attributes.RestaurantsTableService").alias("RestaurantsTableService"), \
col("attributes.RestaurantsTakeOut").alias("RestaurantsTakeOut"), \
col("attributes.Smoking").alias("Smoking"), \
col("attributes.WheelchairAccessible").alias("WheelchairAccessible"), \
col("attributes.WiFi").alias("WiFi"), \
split(col("attributes.BusinessParking"), ":|,").getItem(1).alias("BusinessParking_garage"), \
split(col("attributes.BusinessParking"), ":|,").getItem(3).alias("BusinessParking_street"), \
split(col("attributes.BusinessParking"), ":|,").getItem(5).alias("BusinessParking_validated"), \
split(col("attributes.BusinessParking"), ":|,").getItem(7).alias("BusinessParking_lot"), \
split(col("attributes.BusinessParking"), ":|,|\}").getItem(9).alias("BusinessParking_valet"))
df_raw_business_attributes_json.coalesce(1).write.csv(business_attributes_file_csv_staging_path, mode = 'overwrite', header = 'true')
df_staging_business_attributes_csv = spark.read.csv(business_attributes_file_csv_staging_path, header = 'true')
print ("__________________Raw Schema__________________")
df_staging_business_attributes_csv.printSchema()
print ("__________________Raw Count__________________\n")
print(df_staging_business_attributes_csv.count())
print("\n")
print ("__________________Raw Sample__________________\n")
print(df_staging_business_attributes_csv.head(10))
df_staging_business_attributes_csv.show(5, truncate = False)

StatementMeta(sparkpool, 28, 33, Finished, Available)

__________________Raw Schema__________________
root
 |-- business_id: string (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: s